# Урок 2: Post-Training Weight-Only Quantization

Мы берем обученную модель и сжимаем только веса, оставляя активации в высокой точности. Это самый эффективный способ запустить 70B модель на одной видеокарте.

## 1. Математическая теория

### 1.1. Общая постановка
Цель: $\min_{\hat{W}} \| WX - \hat{W}X \|^2$. 
В отличие от наивного квантования весов, здесь мы учитываем влияние весов на выход слоя через активации $X$.

### 1.2. Методы из обзора:
*   **GPTQ (Frantar et al., 2023):** Использует обратную матрицу Гессиана для коррекции весов. Метод квантует одну строку за раз и обновляет оставшиеся веса, чтобы скомпенсировать ошибку:
    $$\delta w_i = (w_i - \text{quant}(w_i)) \cdot H_{ii}^{-1} \cdot H_{i,:}$$
*   **AWQ (Lin et al., 2023):** Идея в том, что не все веса одинаково важны. Веса, отвечающие за большие значения активаций (выбросы), защищаются через масштабирование (scaling) перед квантованием.
*   **QuIP (Chee et al., 2023):** Применяет некогерентные преобразования (умножение на случайные ортогональные матрицы), чтобы «размазать» выбросы по всей матрице весов, делая её идеальной для равномерного квантования.
*   **SpQR (Dettmers et al., 2024):** Гибридный подход: 99% весов сжимаются агрессивно (3-4 бита), а 1% критически важных выбросов хранятся в FP16 в разреженном формате.

---

In [ ]:
import torch
from src.model import GPTLanguageModel, device

def pseudo_quantize_weight_raw(w, bits=4):
    scale = w.abs().max() / (2**(bits-1) - 1)
    return torch.round(w / (scale + 1e-6)).clamp(-(2**(bits-1)), 2**(bits-1)-1) * scale

print("nanoGPT Track: Реализовано наивное квантование весов через тензорные операции.")

## 2. Промышленная реализация: AutoAWQ
Библиотека **AutoAWQ** автоматизирует процесс поиска масштабов и квантования моделей Llama/Mistral в 4 бита.

In [ ]:
"""
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = 'meta-llama/Llama-2-7b-hf'
model = AutoAWQForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4 }
model.quantize(tokenizer, quant_config=quant_config)
"""
print("Llama Track: AutoAWQ является стандартом для Weight-Only квантования.")